In [1]:
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
import os
os.getcwd()

'/Users/hty/Desktop/ECE1786/GOTalk/train'

In [ ]:
## uncomment to run on colab GPU


# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

In [ ]:
## uncomment to run on colab GPU

# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#   print('Not using a high-RAM runtime')
# else:
#   print('You are using a high-RAM runtime!')

# ! pip install datasets transformers
# !apt install git-lfs

In [ ]:
# login to huggingface

from huggingface_hub import notebook_login

notebook_login()

In [2]:
file = open("../data/got_1+2_fined.txt", "r")
full_texts =[]
texts = []
labels = []
for i in range(1793):
    line = file.readline()
    words = word_tokenize(line)
    sentence = TreebankWordDetokenizer().detokenize(words)
    if len(sentence) == 0: 
        print(line, i)
    if sentence[-1] == ']':
        sentence = sentence + ' '
    full_texts.append(sentence)
    # texts.append(TreebankWordDetokenizer().detokenize(words[:-1]))
    # labels.append(TreebankWordDetokenizer().detokenize(words[1:]))
    
train, test = train_test_split(full_texts, test_size=0.2, shuffle=False)


file = open('../data/got_train.txt', 'w')
for i in train:
    file.write(i)
file.close()

file = open('../data/got_test.txt', 'w')
for i in test:
    file.write(i)
file.close()

In [3]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling

tokenizer = AutoTokenizer.from_pretrained("HScomcom/gpt2-game-of-thrones", truncation=True)

SPECIAL_TOKENS = {
    "bos_token": "[BOS]",
    "eos_token": "[EOS]",
    "pad_token": "[PAD]"
    # "additional_special_tokens": 
}
tokenizer.add_special_tokens(SPECIAL_TOKENS)

train_path = '../data/got_train.txt'
# train_path = '../data/got_1+2_fined.txt'
test_path = '../data/got_test.txt'

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=100,
          )

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=100,
          )   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)



/Users/hty/opt/anaconda3/envs/ECE1786/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [16]:
train_dataset[100]

tensor([  780,   339,   564,   247,   264,  2330,   764,   383,  1854,   389,
          477,  3223,    11, 13791,   393,  2042,   764,   220, 50258,  1318])

In [6]:
tokenizer.decode(train_dataset[99])

' s too … it ’ s cold. [EOS] Yes. Cold and hard and mean, that ’ s the Wall, and the men who walk it. Not like the stories your wet nurse told you. Well, piss on the stories and piss on your wet nurse. This is the way it is, and you ’ re here for life, same as the rest of us.Jon repeated bitterly. The armorer could talk about life. He ’ d had one. He �'

In [7]:
tokenizer

PreTrainedTokenizerFast(name_or_path='HScomcom/gpt2-game-of-thrones', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[BOS]', 'eos_token': '[EOS]', 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '[PAD]'})

In [43]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("HScomcom/gpt2-game-of-thrones")

training_args = TrainingArguments(
    f"GPT2-GOT2-finetuned",
    # output_dir="/content", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 1000, # Number of update steps between two evaluations.
    evaluation_strategy="steps",
    save_steps=1000, # after # steps model is saved 
    warmup_steps=250,# number of warmup steps for learning rate scheduler
    push_to_hub=True # push to the huggingface
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,

)

trainer.train()
# trainer.save_model()

loading configuration file config.json from cache at /Users/hty/.cache/huggingface/hub/models--HScomcom--gpt2-game-of-thrones/snapshots/481371376135f570a8c1a4681ccddede9f305acb/config.json
Model config GPT2Config {
  "_name_or_path": "HScomcom/gpt2-game-of-thrones",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
 

Downloading:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /Users/hty/.cache/huggingface/hub/models--HScomcom--gpt2-game-of-thrones/snapshots/481371376135f570a8c1a4681ccddede9f305acb/pytorch_model.bin
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at HScomcom/gpt2-game-of-thrones.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 2614
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & a

  0%|          | 0/7842 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# evaluate
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
text = "Jon said [BOS]"
encoded_input = tokenizer(text, return_tensors='pt').input_ids
# input_ids = encoded_input.input_ids

outputs = model.generate(encoded_input.to(device), do_sample=True, max_length=200, temperature=1.1, top_p = 0.8, pad_token_id = 50256)
res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(res)

In [ ]:
# upload to huggingface
model.push_to_hub("GPT2-GOT4-finetuned")
tokenizer.push_to_hub("GPT2-GOT4-finetuned")
